In [ ]:
# Cell 1: Load Results and Data
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')

# Load model results
with open(os.path.join('..', 'models', 'model_results.pkl'), 'rb') as f:
    results = pickle.load(f)

model = results['model']
scaler = results['scaler']
feature_columns = results['feature_columns']
metrics = results['metrics']

# Load dataset
dataset = pd.read_csv(os.path.join('..', 'data', 'processed_dataset.csv'))

print("=== Model Results Loaded ===")
print(f"Dataset: {len(dataset)} samples")
print(f"Companies: {dataset['ticker'].nunique()}")
print(f"Features ({len(feature_columns)}): {feature_columns}")
print(f"\nMetrics:")
for k, v in metrics.items():
    print(f"  {k}: {v:.4f}")

In [ ]:
# Cell 2: Model Performance Summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X = dataset[feature_columns].copy()
y = dataset['target_return'].copy()

X_scaled = pd.DataFrame(scaler.transform(X), columns=feature_columns)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

# Direction accuracy
direction_correct = np.sum(np.sign(y_test.values) == np.sign(y_pred_test))
direction_accuracy = direction_correct / len(y_test)

print("="*55)
print("    STOCK RETURN PREDICTION - MODEL PERFORMANCE")
print("="*55)
print(f"\nDataset: {len(dataset)} company-quarter samples")
print(f"Companies: {dataset['ticker'].nunique()}")
print(f"Features: {len(feature_columns)}")
print(f"Train/Test split: {len(X_train)}/{len(X_test)}")
print(f"\n--- Regression Metrics ---")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.4f}")
print(f"Test RMSE:  {np.sqrt(mean_squared_error(y_test, y_pred_test)):.4f}")
print(f"Train R2:   {r2_score(y_train, y_pred_train):.4f}")
print(f"Test R2:    {r2_score(y_test, y_pred_test):.4f}")
print(f"Test MAE:   {mean_absolute_error(y_test, y_pred_test):.4f}")
print(f"\n--- Cross-Validation ---")
print(f"CV RMSE:    {metrics.get('cv_rmse_mean', 0):.4f} (+/- {metrics.get('cv_rmse_std', 0):.4f})")
print(f"\n--- Direction Accuracy ---")
print(f"Correct direction: {direction_correct}/{len(y_test)} ({direction_accuracy:.1%})")
print(f"(Baseline random: 50%)")

In [ ]:
# Cell 3: Feature Importance Analysis
importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance Ranking:")
print("-" * 40)
for _, row in importance.iterrows():
    bar = '#' * int(row['importance'] * 50)
    print(f"{row['feature']:25s} {row['importance']:.3f} {bar}")

# Show top 10 features in chart for readability
top_n = min(15, len(importance))
top_features = importance.head(top_n)

fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(top_features['feature'][::-1], top_features['importance'][::-1], color='steelblue')
ax.set_xlabel('Importance Score')
ax.set_title(f'XGBoost Feature Importance (Top {top_n} of {len(feature_columns)})')
plt.tight_layout()
plt.savefig(os.path.join('..', 'results', 'feature_importance_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 4: Prediction Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 11))

# 1. Predicted vs Actual
axes[0, 0].scatter(y_test, y_pred_test, alpha=0.7, edgecolor='black', s=60)
lims = [min(y_test.min(), y_pred_test.min()) - 0.05, max(y_test.max(), y_pred_test.max()) + 0.05]
axes[0, 0].plot(lims, lims, 'r--', label='Perfect prediction')
axes[0, 0].set_xlabel('Actual Return')
axes[0, 0].set_ylabel('Predicted Return')
axes[0, 0].set_title('Predicted vs Actual Quarterly Returns')
axes[0, 0].legend()

# 2. Residuals
residuals = y_test.values - y_pred_test
axes[0, 1].hist(residuals, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 1].axvline(x=0, color='red', linestyle='--')
axes[0, 1].set_xlabel('Prediction Error')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Prediction Errors')

# 3. Average return by company
company_returns = dataset.groupby('ticker')['target_return'].mean().sort_values()
colors = ['salmon' if v < 0 else 'steelblue' for v in company_returns.values]
axes[1, 0].barh(company_returns.index, company_returns.values, color=colors)
axes[1, 0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Average Quarterly Return')
axes[1, 0].set_title(f'Average Return by Company ({dataset["ticker"].nunique()} companies)')
axes[1, 0].tick_params(axis='y', labelsize=7)

# 4. Top feature correlations with target
correlations = dataset[feature_columns + ['target_return']].corr()['target_return'].drop('target_return').sort_values()
top_corr = pd.concat([correlations.head(8), correlations.tail(8)]).drop_duplicates()
corr_colors = ['salmon' if v < 0 else 'steelblue' for v in top_corr.values]
axes[1, 1].barh(top_corr.index, top_corr.values, color=corr_colors)
axes[1, 1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Correlation with Target Return')
axes[1, 1].set_title('Feature Correlations (Top/Bottom)')

plt.tight_layout()
plt.savefig(os.path.join('..', 'results', 'prediction_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 5: Correlation Heatmap
fig, ax = plt.subplots(figsize=(16, 14))

corr_matrix = dataset[feature_columns + ['target_return']].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(
    corr_matrix, mask=mask, annot=True, fmt='.2f',
    cmap='RdBu_r', center=0, square=True,
    linewidths=0.5, ax=ax, annot_kws={'size': 6}
)
ax.set_title('Feature Correlation Heatmap')
ax.tick_params(axis='x', labelsize=7, rotation=45)
ax.tick_params(axis='y', labelsize=7)
plt.tight_layout()
plt.savefig(os.path.join('..', 'results', 'correlation_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 6: Final Summary
print("="*55)
print("    PROJECT SUMMARY")
print("="*55)
print(f"""
Stock Return Prediction using XGBoost
--------------------------------------
Objective: Predict quarterly stock returns from
           financial fundamentals

Data:
  - {dataset['ticker'].nunique()} S&P 500 companies
  - {len(dataset)} company-quarter samples
  - Price data: Kaggle S&P 500 dataset (2013-2018)
  - Financial data: Real SEC EDGAR quarterly filings

Features ({len(feature_columns)}):
  {', '.join(feature_columns[:13])}
  {', '.join(feature_columns[13:])}

Model: XGBoost Regressor
  - max_depth: 4
  - n_estimators: 200
  - learning_rate: 0.05

Results:
  - Test RMSE: {metrics.get('test_rmse', 0):.4f}
  - Test R2: {metrics.get('test_r2', 0):.4f}
  - CV RMSE: {metrics.get('cv_rmse_mean', 0):.4f} (+/- {metrics.get('cv_rmse_std', 0):.4f})
  - Direction Accuracy: {direction_accuracy:.1%}

Top Features: {', '.join(importance['feature'].head(5).values)}

Key Observations:
  - Direction accuracy ({direction_accuracy:.1%}) well above 50% baseline
  - CV RMSE ({metrics.get('cv_rmse_mean', 0):.4f}) consistent with test RMSE ({metrics.get('test_rmse', 0):.4f})
  - Real SEC financial data provides meaningful signal
  - {len(feature_columns)} features extracted from income, balance sheet, and cash flow

Potential Improvements:
  - Add technical indicators (RSI, MACD, moving averages)
  - Extend price data beyond 2013-2018 for more samples
  - Include more companies with complete SEC filings
  - Try ensemble methods or LSTM models for time series
""")

print("All results saved to results/ folder.")
print("Project complete!")